In [289]:
%matplotlib inline
import numpy as np
import scipy as sp
from itertools import product
import json 
import csv
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [300]:

#dataframe for the finding API, will be merged with the shopping GetSingleItem
df_finding=pd.read_json("./data/completedItems.json")


df_getsingleitem = pd.read_json("./data/getSingleItem.json")
df_getsingleitem=df_getsingleitem.rename(columns = {'ItemID':'itemId'})
#df_getsingleitem.head()


df_UserProfile=pd.read_json("./data/getUserProfile.json")
#df_UserProfile.head()

df_UserProfile.head(1880)



#df_getsingleitem.head(100)

#df = pd.merge(df_finding, df_getsingleitem, on='IDHash', how='outer')
#df.head(200)


,Ack,Build,Errors,FeedbackHistory,Timestamp,User,Version
0,Success,E949_CORE_APILW_17769283_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 0, u'P...",2015-12-07T23:07:04.459Z,"{u'Status': u'Confirmed', u'PositiveFeedbackPe...",949
1,Success,E949_CORE_APILW_17769283_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 0, u'P...",2015-12-07T23:07:05.036Z,"{u'Status': u'Confirmed', u'PositiveFeedbackPe...",949
2,Success,E949_CORE_APILW_17769283_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 0, u'P...",2015-12-07T23:07:05.327Z,"{u'Status': u'Confirmed', u'PositiveFeedbackPe...",949
3,Success,E949_CORE_APILW_17769283_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 0, u'P...",2015-12-07T23:07:06.527Z,"{u'Status': u'Confirmed', u'PositiveFeedbackPe...",949
4,Success,E949_CORE_APILW_17769283_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 0, u'P...",2015-12-07T23:07:07.159Z,"{u'Status': u'Confirmed', u'PositiveFeedbackPe...",949
5,Success,E949_CORE_APILW_17769283_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 0, u'P...",2015-12-07T23:07:07.890Z,"{u'Status': u'Confirmed', u'PositiveFeedbackPe...",949
6,Success,E949_CORE_APILW_17769283_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 1, u'P...",2015-12-07T23:07:08.242Z,"{u'Status': u'Confirmed', u'PositiveFeedbackPe...",949
7,Success,E949_CORE_APILW_17769283_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 0, u'P...",2015-12-07T23:07:08.589Z,"{u'Status': u'Confirmed', u'PositiveFeedbackPe...",949
8,Success,E949_CORE_APILW_17769283_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 0, u'P...",2015-12-07T23:07:08.891Z,"{u'Status': u'Confirmed', u'PositiveFeedbackPe...",949
9,Success,E933_CORE_APILW_17636370_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 0, u'P...",2015-12-07T23:07:09.455Z,"{u'Status': u'Confirmed', u'PositiveFeedbackPe...",933


In [239]:
df_UserProfile.head()

,Ack,Build,Errors,FeedbackHistory,Timestamp,User,Version,Status,PositiveFeedbackPercent,SellerBusinessType,NewUser,UserID,FeedbackRatingStar,FeedbackPrivate,FeedbackScore,FeedbackDetailsURL,SellerItemsURL,RegistrationSite,RegistrationDate
0,Success,E949_CORE_APILW_17769283_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 0, u'P...",2015-12-07T23:07:04.459Z,"{u'Status': u'Confirmed', u'PositiveFeedbackPe...",949,Confirmed,100,Private,False,reuv_litz,Yellow,False,28,http://feedback.ebay.com/ws/eBayISAPI.dll?View...,http://search.ebay.com/?sass=reuv_litz&amp;ht=-1,US,2015-09-08T00:21:52.000Z
1,Success,E949_CORE_APILW_17769283_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 0, u'P...",2015-12-07T23:07:05.036Z,"{u'Status': u'Confirmed', u'PositiveFeedbackPe...",949,Confirmed,100,Undefined,False,jeffbast,Blue,False,57,http://feedback.ebay.com/ws/eBayISAPI.dll?View...,http://search.ebay.com/?sass=jeffbast&amp;ht=-1,US,2002-09-03T18:29:33.000Z
2,Success,E949_CORE_APILW_17769283_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 0, u'P...",2015-12-07T23:07:05.327Z,"{u'Status': u'Confirmed', u'PositiveFeedbackPe...",949,Confirmed,100,Private,False,insainair2,Turquoise,False,168,http://feedback.ebay.com/ws/eBayISAPI.dll?View...,http://search.ebay.com/?sass=insainair2&amp;ht=-1,US,2001-09-04T06:09:45.000Z
3,Success,E949_CORE_APILW_17769283_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 0, u'P...",2015-12-07T23:07:06.527Z,"{u'Status': u'Confirmed', u'PositiveFeedbackPe...",949,Confirmed,100,Private,False,vangx254,Turquoise,False,311,http://feedback.ebay.com/ws/eBayISAPI.dll?View...,http://search.ebay.com/?sass=vangx254&amp;ht=-1,US,2007-01-29T20:26:06.000Z
4,Success,E949_CORE_APILW_17769283_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 0, u'P...",2015-12-07T23:07:07.159Z,"{u'Status': u'Confirmed', u'PositiveFeedbackPe...",949,Confirmed,0,Private,False,baseba11er02,None,False,7,http://feedback.ebay.com/ws/eBayISAPI.dll?View...,http://search.ebay.com/?sass=baseba11er02&amp;...,US,2013-01-07T10:43:53.000Z


In [301]:
"""
Function to take each cell in a column when the cell is actually a dictionary. 
Each value in the dictionary is added to a new column of the dataframe based on its key
""" 

#my code here 
def column_dict_expander(df,column):
    for key in df[column][0].keys():
        templist = []
        print key
        counter = 0
        for i in df.index:
            counter +=1
            dictionary = df[column][i]
            if isinstance(dictionary,dict):
                if key in dictionary.keys():
                    val = dictionary[key]
            else:
                val = 'NaN'
            templist.append(val)
        df[key] = templist
    df = df.drop(column, 1)
    return df

In [302]:
# This is to expand some of the columns that have dicitonaries of information.
# I go into each dictionary and make a new column based on each key in the dictionary
# with the function I defined above 


df_getsingleitem = column_dict_expander(df_getsingleitem,'Seller')
df_UserProfile = column_dict_expander(df_UserProfile,'User')

"""
for key in df_UserProfile['User'][0].keys():
    templist = []
    print key
    for i in df_UserProfile.index:
        if df_UserProfile['Ack'][i] == 'Success':
            val = df_UserProfile['User'][i][key]
        else:
            val = 'NaN'
        templist.append(val)
    df_UserProfile[key] = templist
    
    
    
for key in df_getsingleitem['Seller'][0].keys():
    templist = []
    print key
    counter = 0
    for i in df_getsingleitem.index:
        counter +=1
        if key in df_getsingleitem['Seller'][i].keys():
            if df_getsingleitem['Seller'][i][key] != 'NaN':
                val = df_getsingleitem['Seller'][i][key]
        else:
            val = 'NaN'
        templist.append(val)
    df_getsingleitem[key] = templist
"""

FeedbackRatingStar
UserID
FeedbackScore
PositiveFeedbackPercent
Status
PositiveFeedbackPercent
SellerBusinessType
NewUser
UserID
FeedbackRatingStar
FeedbackPrivate
FeedbackScore
FeedbackDetailsURL
SellerItemsURL
RegistrationSite
RegistrationDate


"\nfor key in df_UserProfile['User'][0].keys():\n    templist = []\n    print key\n    for i in df_UserProfile.index:\n        if df_UserProfile['Ack'][i] == 'Success':\n            val = df_UserProfile['User'][i][key]\n        else:\n            val = 'NaN'\n        templist.append(val)\n    df_UserProfile[key] = templist\n    \n    \n    \nfor key in df_getsingleitem['Seller'][0].keys():\n    templist = []\n    print key\n    counter = 0\n    for i in df_getsingleitem.index:\n        counter +=1\n        if key in df_getsingleitem['Seller'][i].keys():\n            if df_getsingleitem['Seller'][i][key] != 'NaN':\n                val = df_getsingleitem['Seller'][i][key]\n        else:\n            val = 'NaN'\n        templist.append(val)\n    df_getsingleitem[key] = templist\n"

In [303]:
##FOR MERGING THE TWO DATAFRAMES ABOUT ITEM


#turns item ID entries from 1-element lists to just the contents inside
df_finding['itemId'] = df_finding.itemId.apply(lambda r: r[0])
#convert type of item ID entries to ints
df_finding['itemId'] = df_finding.itemId.apply(lambda r: int(r))

df_items = pd.merge(df_finding, df_getsingleitem, on='itemId', how='inner')


In [304]:
## MERGING THE FIRST TWO WITH THE ONE ABOUT THE SELLER

df_total = pd.merge(df_items, df_UserProfile, on='UserID', how='inner')

In [305]:
df_total.head()

,autoPay,charityId,condition,country,discountPriceInfo,galleryPlusPictureURL,galleryURL,globalId,isMultiVariationListing,itemId,listingInfo,location,paymentMethod,postalCode,primaryCategory,productId,returnsAccepted,sellingStatus,shippingInfo,subtitle,title,topRatedListing,viewItemURL,AutoPay,AvailableForPickupDropOff,BestOfferEnabled,BidCount,BuyItNowAvailable,BuyItNowPrice,Charity,ConvertedBuyItNowPrice,ConvertedCurrentPrice,Country,CurrentPrice,DiscountPriceInfo,EligibleForPickupDropOff,EndTime,ExcludeShipToLocation,GalleryURL,GlobalShipping,HandlingTime,HighBidder,HitCount,IntegratedMerchantCreditCardEnabled,ListingStatus,ListingType,Location,LotSize,MinimumToBid,NewBestOffer,PaymentAllowedSite,PaymentMethods,PictureURL,PostalCode,PrimaryCategoryID,PrimaryCategoryIDPath,PrimaryCategoryName,ProductID,Quantity,QuantityAvailableHint,QuantitySold,QuantitySoldByPickupInStore,QuantityThreshold,ReserveMet,ReturnPolicy,SKU,ShipToLocations,Site,StartTime,Storefront,Subtitle,TimeLeft,Title,TopRatedListing,ViewItemURLForNaturalSearch,FeedbackRatingStar_x,UserID,FeedbackScore_x,PositiveFeedbackPercent_x,Ack,Build,Errors,FeedbackHistory,Timestamp,Version,Status,PositiveFeedbackPercent_y,SellerBusinessType,NewUser,FeedbackRatingStar_y,FeedbackPrivate,FeedbackScore_y,FeedbackDetailsURL,SellerItemsURL,RegistrationSite,RegistrationDate
0,[false],NaN,"[{u'conditionId': [u'3000'], u'conditionDispla...",[US],NaN,NaN,[http://thumbs4.ebaystatic.com/m/mg6-FUGNDJgBS...,[EBAY-US],[false],131642735687,"[{u'listingType': [u'Auction'], u'gift': [u'fa...","[South Salem,NY,USA]",[PayPal],[10590],"[{u'categoryId': [u'9355'], u'categoryName': [...",NaN,[false],"[{u'currentPrice': [{u'@currencyId': u'USD', u...","[{u'expeditedShipping': [u'true'], u'shippingT...",NaN,[iPhone 5s-32gb-Space Gray-unlocked],[false],[http://www.ebay.com/itm/iPhone-5s-32gb-Space-...,False,NaN,False,41,NaN,NaN,NaN,NaN,"{u'CurrencyID': u'USD', u'Value': 202.09}",US,"{u'CurrencyID': u'USD', u'Value': 202.09}",NaN,NaN,2015-11-09T01:48:14.000Z,NaN,http://thumbs4.ebaystatic.com/pict/13164273568...,False,2,"{u'FeedbackRatingStar': u'Blue', u'FeedbackPri...",190,False,Completed,Chinese,"South Salem, New York",NaN,"{u'CurrencyID': u'USD', u'Value': 204.59}",False,[],[PayPal],[http://i.ebayimg.com/00/s/OTk5WDc0OQ==/z/lDQA...,10590,9355,15032:9355,Cell Phones & Accessories:Cell Phones & Smartp...,NaN,1,NaN,1,0,NaN,NaN,{u'ReturnsAccepted': u'ReturnsNotAccepted'},NaN,[US],US,2015-11-02T01:48:14.000Z,NaN,NaN,PT0S,iPhone 5s-32gb-Space Gray-unlocked,NaN,http://www.ebay.com/itm/iPhone-5s-32gb-Space-G...,Yellow,jdwein1980,19,100.0,Success,E949_CORE_APILW_17769283_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 0, u'P...",2015-12-07T23:40:31.429Z,949,Confirmed,100,Undefined,False,Yellow,False,19,http://feedback.ebay.com/ws/eBayISAPI.dll?View...,http://search.ebay.com/?sass=jdwein1980&amp;ht=-1,US,2004-03-25T21:44:54.000Z
1,[false],NaN,"[{u'conditionId': [u'3000'], u'conditionDispla...",[US],NaN,NaN,[http://thumbs1.ebaystatic.com/m/mQAr2vyjNsA8_...,[EBAY-US],[false],262126933108,"[{u'listingType': [u'Auction'], u'gift': [u'fa...","[Moody,AL,USA]",[PayPal],[35004],"[{u'categoryId': [u'9355'], u'categoryName': [...",NaN,[false],"[{u'currentPrice': [{u'@currencyId': u'USD', u...","[{u'expeditedShipping': [u'true'], u'shipToLoc...",NaN,[apple iphone 5s 32gb],[false],[http://www.ebay.com/itm/apple-iphone-5s-32gb-...,False,NaN,False,4,NaN,NaN,NaN,NaN,"{u'CurrencyID': u'USD', u'Value': 260.0}",US,"{u'CurrencyID': u'USD', u'Value': 260.0}",NaN,NaN,2015-11-09T01:47:58.000Z,NaN,http://thumbs1.ebaystatic.com/pict/26212693310...,False,3,"{u'FeedbackRatingStar': u'Blue', u'FeedbackPri...",32,False,Completed,Chinese,"Moody, Alabama",NaN,"{u'CurrencyID': u'USD', u'Value': 265.0}",False,[],[PayPal],[http://i.ebayimg.com/00/s/MTAwMFg3NTA=/z/djoA...,35004,9355,15032:9355,Cell Phones & Accessories:Cell Phones & Smartp...,NaN,1,NaN,1,0,NaN,NaN,{u'ReturnsAccepted': u'ReturnsNotAccepted'},NaN,[US],US,2015-11-04T01:47:5

**NEXT, we will expad some more columns and delete others that we just don't need to do our analysis. This could even come at the end once we figure out which aren't useful, but must remember to put it here.**



## Now we will do some Feature engineering

This is to figure out which variables actually make a distinction on profitability

In [306]:
df_total.shape


(9629, 96)

We delete the columns that are irrelevant to our analysis. For example, DiscountPriceInfo tells us no information about our data, because none of our auctions have discounted prices.

In [266]:
del df_getsingleitem['Charity'], df_getsingleitem['Country'], df_getsingleitem['ConvertedCurrentPrice'], df_getsingleitem['DiscountPriceInfo']
del df_getsingleitem['ListingStatus'], df_getsingleitem['PaymentAllowedSite'], df_getsingleitem['ReserveMet'], df_getsingleitem['SKU'], df_getsingleitem['Site'], df_getsingleitem['Storefront']
del df['Subtitle'], df['TimeLeft'], df['ViewItemURLForNaturalSearch'], df['ConvertedBuyItNowPrice']

We then remove unnecessary information from each of the columns. For example, on CurrentPrice, we only want the value in the dictionary and not the whole dictionary. 

In [267]:
#df['CurrentPrice'].values[3][u'Value']

df['ListingType'] = df['ListingType'].map({'Chinese': 0, 'FixedPriceItem': 1})
df['paymentMethod'] = df['paymentMethod'].map({'[PayPal]': 0, ''})
df=df.rename(columns = {'CurrentPrice':'FinalPrice'})

for i in df.index:
    
    #getting just the float value and ridding of the dictionary 
    df['CurrentPrice'].values[i] = df['CurrentPrice'].values[i][u'Value']
    
    if pd.isnull(df['BuyItNowPrice'].values[i]) == False:
        #print df['BuyItNowPrice'].values[i]
        df['BuyItNowPrice'].values[i] = df['BuyItNowPrice'].values[i][u'Value']
    if pd.isnull(df['MinimumToBid'].values[i]) == False:
        df['MinimumToBid'].values[i] = df['MinimumToBid'].values[i][u'Value']
        
    
#because each auction we are dealing with is completed, we say
df=df.rename(columns = {'CurrentPrice':'FinalPrice'})

SyntaxError: invalid syntax (<ipython-input-267-2a2db1e8cf48>, line 4)

In [ ]:
df.shape

In [ ]:
dftouse = df.copy()

**We can now make some separator variables to make listings unique**

Certain parameters that are zero are can be significant indicators of specific characteristics of an auction. For example, if the HitCount is zero, that means nobody really bothered to interact with the auction at all.

In [ ]:
ZERO_IMPORTANT_VARIABLES = ['BidCount', 'HitCount']

To mark binary or indicator variables, we see which of the dataframe fall into this category

In [ ]:
for v in df.columns:
    l=df[v].unique()
    if len(l) <= 10:
        print v, l

In [ ]:
json1 = open('./data/completedItems.json', 'r')
json2 = open('./data/getSingleItem.json', 'r')

doc1 = json.load(json1)
doc2 = json.load(json2)
newItemIdSet = set()
counter = 0
for auc in doc1:
    newItemIdSet.add(auc['itemId'][0])
for auc in doc2:
    if auc['ItemID'] not in newItemIdSet:
        counter += 1
print counter
json1.close()
json2.close()